In [2]:
from persper.analytics.cpp import CPPGraphServer
from persper.analytics.analyzer import Analyzer
from persper.analytics.graph_server import CPP_FILENAME_REGEXES
import pickle

ALPHA = 0.85


In [3]:
def is_merged_commit(commit):
    if len(commit.parents) > 1:
        return True
    else:
        return False

def share_distribution(commits, commit_share):
    print(commit_share)
    shares = {}
    for commit in commits:
        if commit.hexsha in commit_share:
            if commit.authored_date in shares:
                shares[commit.authored_date] += commit_share[commit.hexsha]
            else:
                shares[commit.authored_date] = commit_share[commit.hexsha]
    
    shares = dict(sorted(shares.items()))
    
    print(shares.keys())
#     init_commit = shares.keys()[0]
#     last_commit = shares.keys()[-1]
    
    print(init_commit)
    print(last_commit)

    values = []
    for date in interval(init_commit, last_commit):
        if shares[date]:
            values.appned(shares[date])
        else:
            values.appned(0)

    return init_commit, last_commit, values

def developer_profile(pickle_path, alpha=0.85, show_merge=True):
    dev_share = {}
    az = pickle.load(open(pickle_path, 'rb'))

    commit_share = az.get_graph().commit_devranks(alpha, black_set=[])

    for commit in az._ri.repo.iter_commits():

        if is_merged_commit(commit) and not show_merge:
            continue

        email = commit.author.email

        if email in dev_share:
            # dev_share[email]["share"] += commit_share
            dev_share[email]["commits"].append(commit)
        else:
            dev_share[email] = {}
            dev_share[email]["commits"] = [commit]
            dev_share[email]["email"] = email
            # dev_share[email]["share"] = commit_share

    for values in dev_share.values():
        init_commit, last_commit, values = share_distribution(values['commits'], commit_share)
        values['distribution'] = {
            'init_commit': init_commit,
            'last_commit': last_commit,
            'values': values
        }
        values['dev_value'] = sum(values)

    return dev_share

In [4]:
pickle_path = '/Users/xu.b.zheng/pickles/zhengxu001/bitcoin/1550271482.pickle'

In [5]:
dev_share = developer_profile(pickle_path)

{}
dict_keys([])


NameError: name 'init_commit' is not defined